In [2]:
import azurescript as azs

azs.ConnectToAzure()
# # azs.UploadData()
# azs.CreateAzureDataset()
# azs.PrepareAzureScript()
# azs.RunAzureScript()
azs.SaveModel()
print('done')

Workspace: AzureML-Pytorch
Compute Target: AzPytrch-NC6
Datastore: workspaceblobstore
Experiment: 616_Final
LAST RUN ID:  616_Final_1621370965_7bf7ff92
Model 'ANFIS-PyTorch' version 11 registered 
done


In [5]:
## LOCAL TEST MODEL
import torch
import scripts.model as mod
import scripts.preprocess as pp
import torch.nn as nn
from torch import optim
from torch.nn.modules.activation import Sigmoid





C:\Users\gkurp\.conda\envs\AzurePytorch\lib\site-packages\torch\nn\modules\loss.py:96: UserWarning: Using a target size (torch.Size([100])) that is different to the input size (torch.Size([100, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
C:\Users\gkurp\.conda\envs\AzurePytorch\lib\site-packages\torch\nn\modules\loss.py:96: UserWarning: Using a target size (torch.Size([66])) that is different to the input size (torch.Size([66, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
C:\Users\gkurp\.conda\envs\AzurePytorch\lib\site-packages\torch\nn\modules\loss.py:96: UserWarning: Using a target size (torch.Size([34])) that is different to the input size (torch.Size([34, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure 

In [1]:
##LOCAL TEST PREPRPOCESS

import scripts.preprocess as pp
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
datapath = "./data/"
master, payments = pp.LoadData(datapath, filename='alldone.csv')
print('Data Loaded')
solution, master = pp.PrepMLPData(master, payments)
print('Solution String Created')
train_dl, test_dl = pp.CreateMLPDataLoader(master, solution)
print('Data Loader Created')

filename = os.path.join(datapath, 'train_dl.pt')
torch.save(train_dl, 'train_dl.pt')
filename = os.path.join(datapath, 'test_dl.pt')
torch.save(test_dl, 'test_dl.pt')
print('YAY!')

Data Loaded
Solution String Created
Data Loader Created
YAY!


In [5]:
n = 0
for x, y in train_dl:
    print(x)
    n += 1
    if n > 1: break

tensor([[8.0000e+00, 2.1101e+05, 3.0857e+05,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [8.0000e+00, 2.2596e+05, 3.2309e+05,  ..., 0.0000e+00, 1.0000e+00,
         1.0000e+00],
        [8.0000e+00, 1.8101e+05, 2.7874e+05,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        ...,
        [8.0000e+00, 1.8083e+05, 2.7855e+05,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [8.0000e+00, 1.9975e+05, 2.9737e+05,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [8.0000e+00, 2.4305e+05, 3.4004e+05,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00]], dtype=torch.float64)
tensor([[8.0000e+00, 1.9525e+05, 2.9291e+05,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [8.0000e+00, 1.9755e+05, 2.9520e+05,  ..., 1.0000e+00, 0.0000e+00,
         0.0000e+00],
        [8.0000e+00, 2.2625e+05, 3.2337e+05,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        ...,
        [8.0000e+00, 2.1404e+05, 3.1160e+05,  ..., 0.0000e+00, 0.0000e+00,
    

In [15]:
master[master['accountid']==24837]

,clientname,accountid,customerid,ssnlastfour,Address1,Address2,City,State,PostalCode,dateplaced,...,placeddpd,hasbrokenpromise,hasconfirmed,haskeptptp,haspayments,hasplacementphone,haspossiblenumber,haspromever,Month1,Month2
